Đồ án cuối kỳ
Thành viên:
Họ tên: Trần Phúc Nguyên
MSSV: 18120488
Họ tên: Trần Thành Long
MSSV: 18120455

## Import các thư viện cần thiết

In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


---

## Thu thập dữ liệu

Trong bài này, ta sẽ dùng dữ liệu đã được thu thập sẵn là file "vgsales.csv" đính kèm; đây là dữ liệu về danh sách các video game đã bán hơn 100,000 bản("vgsales" là viết tắt của "video games sales"). Dữ liệu này được lấy từ trang Kaggle link trang:(https://www.kaggle.com/gregorut/videogamesales). \
Lần cuối dữ liệu được update là 26/10/2016.
Dữ liệu được thu thập bằng cách sử dụng BeautifulSoup của Python(link script: https://github.com/GregorUT/vgchartzScrape.) để cào dữ liệu từ website vgchartz.com.
Dữ liệu không có License.

---

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [2]:
vgsales_df = pd.read_csv('vgsales.csv')
vgsales_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Dữ liệu gồm có bao nhiêu dòng và bao nhiêu cột?

In [3]:
num_rows, num_cols = vgsales_df.shape
num_rows, num_cols

(16598, 11)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Theo mô tả dữ liệu của nguồn từ trang Kaggle và theo quan sát sơ bộ về dữ liệu thì một dòng cho biết thông tin về sales của một tựa game nào đó. Có vẻ không có dòng nào bị "lạc loài".

### Dữ liệu có các dòng bị lặp không?

In [4]:
vgsales_df.duplicated().any()

False

### Mỗi cột có ý nghĩa gì?

Dưới đây là phần mô tả của nguồn từ trang Kaggle về các cột trong file "vgsales.csv"

- **Rank:** Ranking of overall sales

- **Name:** The games name

- **Platform:** Platform of the games release (i.e. PC,PS4, etc.)

- **Year:** Year of the game's release

- **Genre:** Genre of the game

- **Publisher:** Publisher of the game

- **NA_Sales:** Sales in North America (in millions)

- **EU_Sales:** Sales in Europe (in millions)

- **JP_Sales:** Sales in Japan (in millions)

- **Other_Sales:** Sales in the rest of the world (in millions)

- **Global_Sales:** Total worldwide sales.

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không? 

In [5]:
vgsales_df.dtypes

Rank              int64
Name             object
Platform         object
Year            float64
Genre            object
Publisher        object
NA_Sales        float64
EU_Sales        float64
JP_Sales        float64
Other_Sales     float64
Global_Sales    float64
dtype: object

`vgsales_df` có 4 cột có `dtype` là `object`: "Name", "Platform", "Genre", "Publisher". Trong HW3, chúng ta đã hiểu về ý nghĩa của `object` trong Pandas. Chúng ta cũng sẽ viết một hàm nhận đầu vào là một cột `s` (series) có `dtype` là `object` và trả về tập các kiểu dữ liệu thật sự có trong cột này.

In [6]:
def open_object_dtype(s):
    dtypes = set()
    s.apply(lambda x: dtypes.add(type(x)))
    return dtypes

In [7]:
print(open_object_dtype(vgsales_df['Name']))
print(open_object_dtype(vgsales_df['Platform']))
print(open_object_dtype(vgsales_df['Genre']))
print(open_object_dtype(vgsales_df['Publisher']))

{<class 'str'>}
{<class 'str'>}
{<class 'str'>}
{<class 'str'>, <class 'float'>}


Chúng ta sẽ kiểm tra xem tại sao lại có các gìá trị `float` ở cột `Year` và cột `Publisher`. Dự đoán có thể là do sự tồn tại của các giá trị NaN ở các cột này. Chúng ta sẽ kiểm tra thử số lượng các giá trị NaN của các cột.

In [31]:
vgsales_df.isna().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

Như vậy dự đoán của chúng ta là chính xác, sự xuất hiện của các giá trị NaN ở cột `Year` và `Publisher` có thể là do các tựa game này là một các tựa game nhỏ được phát triển bởi vài cá nhân, nên chúng ta không có thông tin chi tiết về.

**Vấn đề cần tiền xử lý:**
- Cột `Year` không nhất thiết phải chuyển về `int`, nên chúng ta sẽ tạm sử dụng dtype `float` vì có sự xuất hiện của các giá trị NaN.
- Cột "Publisher" có dtype gồm 2 loại là `str` và `float`. Nếu kiểm tra thêm thì chúng ta sẽ thấy `float` là ứng với `NaN`, còn `str` là ứng với tên của nhà phát hành.  Để có thể tiếp tục khám phá thêm về cột này, ta cũng sẽ thực hiện bước tiền xử lý là chuyển tất cả sang dtype là `str`. Những cột có giá trị NaN sẽ được để khoảng trống.

---

## Tiền xử lý

### Chuyển dtype của cột `Publisher` về str:

Chúng ta sẽ fill các giá trị NaN bằng `''`, sau đó sử dụng `astype` để đưa type thực sự về `str`.

In [42]:
vgsales_df['Publisher'] = vgsales_df['Publisher'].fillna('').astype('str')
open_object_dtype(vgsales_df['Publisher'])

{str}

---

## Khám phá dữ liệu (tiếp tục)

### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

#### Định nghĩa hàm *`missing_ratio`*:

In [51]:
def missing_ratio(s):
    return s.isna().mean() * 100

Hiện có 6 cột có vẻ là thuộc nhóm numeric: `Rank`, `NA_Sales`, `EU_Sales`, `JP_Sales`, `Other_Sales`, `Global_Sales`. Tuy nhiên, cột `Rank` có vẻ thuộc nhóm categorical. Như vậy, chỉ có 5 cột numeric là `NA_Sales`, `EU_Sales`, `JP_Sales`, `Other_Sales`, `Global_Sales`. 

Với mỗi cột numeric chúng ta sẽ tính tỉ lệ % giá trị thiếu (từ 0 đến 100), min, max. Dataframe kết quả có 3 dòng là "missing_ratio", "min", "max"; và có 5 cột là `NA_Sales`, `EU_Sales`, `JP_Sales`, `Other_Sales`, `Global_Sales`.

In [52]:
nume_col_profiles_df = vgsales_df[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"]].agg(\
    [missing_ratio, 'min', 'max'])
nume_col_profiles_df

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
missing_ratio,0.00,0.00,0.00,0.00,0.00
min,0.00,0.00,0.00,0.00,0.01
max,41.49,29.02,10.22,10.57,82.74


Các cột đều không có giá trị thiếu. Min và max của các cột có vẻ không có gì bất thường.

### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

In [55]:
vgsales_df.columns

Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')

`vgsales_df` có 6 cột categorical là `Rank`, `Name`, `Platform`, `Year`, `Genre`, `Publisher`.

Với mỗi cột categorical, chúng ta sẽ tính tỉ lệ % giá trị thiếu (từ 0 đến 100), số lượng giá trị khác nhau, list/array một vài các giá trị khác nhau. Dataframe kết quả có 3 dòng là "missing_ratio", "num_diff_vals", "some_diff_vals"; và có 6 cột là `Rank`, `Name`, `Platform`, `Year`, `Genre`, `Publisher`.

In [57]:
# YOUR CODE HERE
cate_col_profiles_df = vgsales_df[['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher']].agg(\
    [missing_ratio, 'nunique', 'unique'])
cate_col_profiles_df = cate_col_profiles_df.rename(index={'nunique':'num_diff_vals', 'unique':'some_diff_vals'})
cate_col_profiles_df

,Rank,Name,Platform,Year,Genre,Publisher
missing_ratio,0.0,0.0,0.0,1.632727,0.0,0.0
num_diff_vals,16598,11493,31,39,12,579
some_diff_vals,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[Wii Sports, Super Mario Bros., Mario Kart Wii...","[Wii, NES, GB, DS, X360, PS3, PS2, SNES, GBA, ...","[2006.0, 1985.0, 2008.0, 2009.0, 1996.0, 1989....","[Sports, Platform, Racing, Role-Playing, Puzzl...","[Nintendo, Microsoft Game Studios, Take-Two In..."


---

## Đưa ra các câu hỏi cần trả lời